In [1]:
import numpy as np



In [2]:
drop_cols = ["sourcePayloadAsBase64", "sourcePayloadAsUTF", 
                 "destinationPayloadAsBase64", "destinationPayloadAsUTF"]
import pandas as pd
filepath='TestbedSunJun13Flows.csv'    
df = pd.read_csv(filepath)
df = df.drop_duplicates()
df = df.drop(columns=drop_cols, errors='ignore')

In [3]:
df

,generated,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,source,protocolName,sourcePort,destination,destinationPort,startDateTime,stopDateTime,Label
0,3/11/2014 17:35,Unknown_UDP,2633658,0,0,28971,L2R,NaN,NaN,192.168.5.122,udp_ip,5353,224.0.0.251,5353,6/12/2010 23:57,6/13/2010 9:24,Normal
2,3/11/2014 17:35,HTTPWeb,64,128,2,1,L2L,"F,A",R,192.168.2.113,tcp_ip,4191,192.168.5.122,80,6/12/2010 23:57,6/12/2010 23:59,Normal
4,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.113,tcp_ip,4192,207.241.148.80,80,6/12/2010 23:57,6/12/2010 23:59,Normal
6,3/11/2014 17:35,HTTPImageTransfer,128,128,2,2,L2R,"F,A","F,A",192.168.2.110,tcp_ip,1864,216.49.88.12,80,6/12/2010 23:57,6/12/2010 23:59,Normal
8,3/11/2014 17:35,SecureWeb,64,0,0,1,L2R,"F,A",NaN,192.168.2.113,tcp_ip,4186,63.245.209.72,443,6/12/2010 23:58,6/12/2010 23:59,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275517,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.111,tcp_ip,4430,72.14.204.100,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275520,3/11/2014 17:35,HTTPWeb,128,64,1,2,L2R,"F,A","F,A",192.168.2.111,tcp_ip,4428,125.89.196.18,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275522,3/11/2014 17:35,HTTPWeb,256,0,0,4,L2R,"F,A",NaN,192.168.2.111,tcp_ip,4429,69.163.132.48,80,6/13/2010 23:58,6/13/2010 23:58,Normal
275523,3/11/2014 17:35,HTTPWeb,128,0,0,2,L2R,"F,A",NaN,192.168.2.111,tcp_ip,4431,69.163.132.48,80,6/13/2010 23:58,6/13/2010 23:58,Normal


In [4]:
print(df['Label'].value_counts())

Label
Normal    126430
Attack     10139
Name: count, dtype: int64


In [5]:
import pandas as pd
from sklearn.utils import resample


# Check the unique counts in the label column
print(df['Label'].value_counts())

# Separate classes
class_counts = df['Label'].value_counts()
minority_class = class_counts.idxmin()  # Class with fewer samples
majority_class = class_counts.idxmax()  # Class with more samples


df_minority = df[df['Label'] == minority_class]
df_majority = df[df['Label'] == majority_class]


# Undersample both classes to 70 records each
df_minority_undersampled = resample(df_minority, 
                                    replace=False, 
                                    n_samples=2000, 
                                    random_state=42)

df_majority_undersampled = resample(df_majority, 
                                    replace=False, 
                                    n_samples=2000, 
                                    random_state=42)

# Combine the undersampled data
df_balanced = pd.concat([df_minority_undersampled, df_majority_undersampled])

# Shuffle the dataset to mix the classes
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new class distribution
print(df_balanced['Label'].value_counts())




Label
Normal    126430
Attack     10139
Name: count, dtype: int64
Label
Attack    2000
Normal    2000
Name: count, dtype: int64


In [6]:
df=df_balanced
df.drop(columns=['generated', 'startDateTime', 'stopDateTime', 'source', 'destination'], inplace=True)


In [7]:
df

,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,protocolName,sourcePort,destinationPort,Label
0,Unknown_TCP,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,15742,Attack
1,NetBIOS-IP,247,0,0,1,L2L,NaN,NaN,udp_ip,138,138,Normal
2,MiscApplication,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,85,Attack
3,HTTPWeb,375,1225,5,5,L2R,"F,S,P,A","F,S,P,A",tcp_ip,4646,80,Normal
4,HTTPImageTransfer,569,5844,7,7,L2R,"F,S,P,A","F,S,P,A",tcp_ip,2985,80,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,Unknown_TCP,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,2381,Attack
3996,Unknown_TCP,128,128,2,2,L2L,S,"R,A",tcp_ip,34431,5009,Attack
3997,MiscApplication,64,64,1,1,L2L,S,"R,A",tcp_ip,54806,3,Attack
3998,HTTPImageTransfer,1979,22787,23,22,L2R,"F,S,P,A","F,S,P,A",tcp_ip,1707,80,Normal


In [8]:

from sklearn.preprocessing import LabelEncoder
import pandas as pd

# List of binary categorical columns
binary_cols = ['direction', 'sourceTCPFlagsDescription', 'destinationTCPFlagsDescription', 'Label','appName', 'protocolName']
le = LabelEncoder()

# Apply Label Encoding for binary categorical features
for col in binary_cols:
    df[col] = le.fit_transform(df[col])





In [9]:
df

,appName,totalSourceBytes,totalDestinationBytes,totalDestinationPackets,totalSourcePackets,direction,sourceTCPFlagsDescription,destinationTCPFlagsDescription,protocolName,sourcePort,destinationPort,Label
0,67,64,64,1,1,0,13,9,2,54806,15742,0
1,42,247,0,0,1,0,17,14,3,138,138,1
2,38,64,64,1,1,0,13,9,2,54806,85,0
3,17,375,1225,5,5,1,6,5,2,4646,80,1
4,16,569,5844,7,7,1,6,5,2,2985,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,67,64,64,1,1,0,13,9,2,54806,2381,0
3996,67,128,128,2,2,0,13,9,2,34431,5009,0
3997,38,64,64,1,1,0,13,9,2,54806,3,0
3998,16,1979,22787,23,22,1,6,5,2,1707,80,1


In [10]:
import numpy as np


from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

x_tr, x_test, y_tr, y_test = train_test_split(X, y, train_size = 0.7,random_state=42)

from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

x_test = scaler.transform(x_test)
x_test = np.clip(x_test,0,1)




In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   appName                         4000 non-null   int64
 1   totalSourceBytes                4000 non-null   int64
 2   totalDestinationBytes           4000 non-null   int64
 3   totalDestinationPackets         4000 non-null   int64
 4   totalSourcePackets              4000 non-null   int64
 5   direction                       4000 non-null   int64
 6   sourceTCPFlagsDescription       4000 non-null   int64
 7   destinationTCPFlagsDescription  4000 non-null   int64
 8   protocolName                    4000 non-null   int64
 9   sourcePort                      4000 non-null   int64
 10  destinationPort                 4000 non-null   int64
 11  Label                           4000 non-null   int64
dtypes: int64(12)
memory usage: 375.1 KB


In [12]:
import pennylane as qml
nqubits = 11
dev = qml.device("default.qubit", wires=nqubits)
@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires = range(nqubits))

In [13]:
from sklearn.svm import SVC
def qkernel(A, B):
    print(A.shape)
    print(B.shape)
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

def qkernel_optimised(A, B):
    """
    Compute the Gram matrix using kernel_circ.
    
    If A and B are square (same number of rows, and identical arrays), then only the
    lower triangle (including diagonal) is computed, and the upper triangle is filled
    by symmetry.
    
    If A and B are not square, the full matrix is computed.
    """
    print(A.shape)
    print(B.shape)
    # Check if both inputs have the same number of rows.
    if A.shape[0] == B.shape[0]:
        # Optionally, you can check if A and B are identical.
        if np.array_equal(A, B):
            n = A.shape[0]
            # Initialize an empty Gram matrix.
            K = np.zeros((n, n))
            # Compute only the diagonal and lower triangle.
            for i in range(n):
                for j in range(i + 1):
                    value = kernel_circ(A[i], B[j])[0]
                    K[i, j] = value
                    if i != j:
                        K[j, i] = value  # Mirror the lower triangle to the upper triangle.
            return K

    # For non-square inputs or non-identical A and B, compute the full matrix.
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])



from joblib import Parallel, delayed

def qkernel_optimised_2(A, B):
    """
    Compute the Gram matrix using kernel_circ in parallel.
    
    If A and B are square (same number of rows, and identical arrays), then only the
    lower triangle (including diagonal) is computed, and the upper triangle is filled
    by symmetry.
    
    If A and B are not square, the full matrix is computed.
    """
    # Check if both inputs have the same number of rows.
    if A.shape[0] == B.shape[0] and np.array_equal(A, B):
        n = A.shape[0]
        K = np.zeros((n, n))
        
        # Define a function to compute a single kernel value for the pair (i, j)
        def compute_pair(i, j):
            value = kernel_circ(A[i], A[j])[0]
            return i, j, value

        # Create list of index pairs for lower triangle (including diagonal)
        indices = [(i, j) for i in range(n) for j in range(i + 1)]
        
        # Compute kernel values in parallel
        results = Parallel(n_jobs=-1)(delayed(compute_pair)(i, j) for i, j in indices)
        
        # Fill in the Gram matrix symmetrically
        for i, j, value in results:
            K[i, j] = value
            if i != j:
                K[j, i] = value
        return K

    else:
        # For non-square inputs or non-identical A and B, compute the full matrix.
        def compute_row(a):
            return [kernel_circ(a, b)[0] for b in B]
        
        K = Parallel(n_jobs=-1)(delayed(compute_row)(a) for a in A)
        return np.array(K)






In [14]:
import time
from sklearn.svm import SVC

# Start the timer
start_time = time.time()

# Fit the SVM model
svm = SVC(kernel=qkernel_optimised_2).fit(x_tr, y_tr)

# Calculate and print the fit time
fit_time = time.time() - start_time
print(f"Fit time: {fit_time:.4f} seconds")


Fit time: 809.1270 seconds


In [15]:
from sklearn.metrics import classification_report,accuracy_score

# Start the timer
start_time = time.time()

# Make predictions
y_pred = svm.predict(x_test)


fit_time = time.time() - start_time
print(f"Predict time: {fit_time:.4f} seconds")

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


KeyboardInterrupt: 

In [ ]:
print("x_tr shape:", x_tr.shape)  # Should be (samples, features)
print("y_tr shape:", y_tr.shape)  # Should be (samples,)

In [ ]:
import pickle
# Save the model to a file
with open('QSVM_ANG_4K.pkl', 'wb') as file:
    pickle.dump(svm, file)